# X-GENRE classifier - multilingual text genre classifier 

https://huggingface.co/classla/xlm-roberta-base-multilingual-text-genre-classifier

Náš model by měl na základě charakteru textu rozpoznat, zda se jedná o jednu z následující kategorií (charakter jednotlivých kategorií viz odkaz výše). Naše data pocházejí z Wikipedie a každý řádek obsahuje úvodní odstavec o nějaké české či zahraniční osobnosti, očekáváme tedy výstupy modelu v kategorii 'Information/Explanation'.


If you use the model, please cite the paper:
@article{kuzman2023automatic,
  title={Automatic Genre Identification for Robust Enrichment of Massive Text Collections: Investigation of Classification Methods in the Era of Large Language Models},
  author={Kuzman, Taja and Mozeti{\v{c}}, Igor and Ljube{\v{s}}i{\'c}, Nikola},
  journal={Machine Learning and Knowledge Extraction},
  volume={5},
  number={3},
  pages={1149--1175},
  year={2023},
  publisher={MDPI}
}

In [2]:
# nacteni dat
from datasets import load_dataset
import numpy as np

dataset = load_dataset("kotlarska2/osobnosti")

In [3]:
# nacteni modelu
from simpletransformers.classification import ClassificationModel
model_args= {
            "num_train_epochs": 15,
            "learning_rate": 1e-5,
            "max_seq_length": 512,
            "silent": True
            }
model = ClassificationModel(
    "xlmroberta", "classla/xlm-roberta-base-multilingual-text-genre-classifier", use_cuda=True,
    args=model_args
    
)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/simpletransformers/classification/classification_model.py:484: UserWarning: use_multiprocessing automatically disabled as xlmroberta fails when using multiprocessing for feature conversion.
  warnings.warn(


In [5]:
# vyhodnoceni modelu
pred, log = model.predict(dataset['train']['text'])

In [22]:
# zastoupeni kategorii v nasich datech
pred_char = [model.config.id2label[i] for i in pred]

unique_values, counts = np.unique(pred_char, return_counts=True)

for value, count in zip(unique_values, counts):
    print(f"Hodnota {value} se vyskytla {count}x.")


Hodnota Information/Explanation se vyskytla 1953x.
Hodnota News se vyskytla 9x.
Hodnota Opinion/Argumentation se vyskytla 1x.
Hodnota Other se vyskytla 1x.
Hodnota Promotion se vyskytla 5x.


In [23]:
# texty, ktere byly zarazenem jinam, nez jsme ocekavali
texty = dataset['train']['text']

# pomocna promenna
mask = (np.array(pred_char) != 'Information/Explanation')

# vyselektovani spatne zarazenych textu
filtered_values = np.array(texty)[mask]

print(filtered_values)

['2007  mistr světa\n' 'Další úspěchy\n' 'LOH 2016 – 26. místo\n'
 '2. místo na ME 2017\n' 'Etapové závody\n' 'Grand Tours\n'
 'Grand Tours\n' 'manželka Veronika\n' 'Etapové závody\n'
 'Etapové závody\n' 'Elizaveta Zhuk (od 2019 - 2022)\n'
 'LOH 2012 – 15. místo\n'
 'LOH 2008 – 13. místo, 23. místoLOH 2012 – 33. místo, 34. místo, 41. místo\n'
 '3. místo – Mistrovství Evropy 2015 v Baku\n'
 'Lenka Prášilová, pod přezdívkou jako Laura Lion, ale vystupuje i pod dalšími jmény, je česká pornoherečka.\n'
 'Eva Šormová\nNoemi Smolik\n']


Ve většině případů jsme dostali očekávanou kategorii 'Information/Explanation', v 16 případech jsme dostali kategorii odlišnou.

-> V 15 z nich se jedná o chybu v datech (špatně načtená data při tvorbě datasetu).

-> V 1 případě se jedná o chybu modelu.

Pro úplnost dodejme, že v předpokladech modelu je vstup textu o délce alespoň 75 slov, přičemž tento požadavek není ve všech našich vstupních textech splněn.